# US Power Outage Analysis

**Name(s)**: Layth Marabeh, Khanh Phan, Danny Xia   
**Repository Link**: https://github.com/k-phantastic/US-Power-Outage-Analysis   
**Website Link**: https://github.com/k-phantastic/US-Power-Outage-Analysis (Github Pages to be updated)

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from utils import * 

# For widescreen display, overrides utils.py settings
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

## Step 1: Introduction

* Understand the data you have access to. Brainstorm a few questions that interest you about the dataset. Pick one question you plan to investigate further. (As the data science lifecycle tells us, this question may change as you work on your project.)

# CHECKPOINT 1: 
(2 points) Which of the three datasets did you choose? Why?

# Dataset Information from Purdue University 

**Source:** https://engineering.purdue.edu/LASCI/research-data/outages/outage.xlsx  
**Data Dictionary:** https://www.sciencedirect.com/science/article/pii/S2352340918307182?via%3Dihub#t0005

>This dataset includes the major outages witnessed by different states in the continental U.S. Besides major outages, this data contains information on geographical location of the outages, regional climatic information, land-use characteristics, electricity consumption patterns and economic characteristics of the states affected by the outages. 

> Column information is located in Table 1, Variable descriptions of the article

In [3]:
# Load the raw dataset
file_path = 'data/outage.xlsx'

raw_df = pd.read_excel(file_path)
raw_df.head(10)

,Major power outage events in the continental U.S.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56
0,Time period: January 2000 - July 2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Regions affected: Outages reported in this dat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,variables,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
5,Units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,numeric,NaN,"Day of the week, Month Day, Year",Hour:Minute:Second (AM / PM),"Day of the week, Month Day, Year",Hour:Minute:Second (AM / PM),NaN,NaN,NaN,mins,Megawatt,NaN,cents / kilowatt-hour,cents / kilowatt-hour,cents / kilowatt-hour,cents / kilowatt-hour,Megawatt-hour,Megawatt-hour,Megawatt-hour,Megawatt-hour,%,%,%,NaN,NaN,NaN,NaN,%,%,%,USD,USD,fraction,%,USD,USD,%,%,NaN,%,%,persons per square mile,persons per square mile,persons per square mile,%,%,%,%,%
6,NaN,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,NaN,NaN,3060,NaN,70000,11.6,9.18,6.81,9.28,2332915,2114774,2113291,6562520,35.55,32.23,32.2,2308736,276286,10673,2595696,88.94,10.64,0.41,51268,47586,1.08,1.6,4802,274182,1.75,2.2,5348119,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
7,NaN,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,vandalism,NaN,1,NaN,NaN,12.12,9.71,6.49,9.28,1586986,1807756,1887927,5284231,30.03,34.21,35.73,2345860,284978,9898,2640737,88.83,10.79,0.37,53499,49091,1.09,1.9,5226,291955,1.79,2.2,5457125,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
8,NaN,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,heavy wind,NaN,3000,NaN,70000,10.87,8.19,6.07,8.15,1467293,1801683,1951295,5222116,28.1,34.5,37.37,2300291,276463,10150,2586905,

In [4]:
# Initial file has the header in row 5, with first column being blank and second column being index
df = pd.read_excel(file_path, header=5, usecols=range(2, 57), )

# Skip the units row
df = df.drop(index=0)
df.head()

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,NaN,NaN,3060,NaN,70000.0,11.6,9.18,6.81,9.28,2332915,2114774,2113291,6562520,35.55,32.23,32.2,2.31e+06,276286.0,10673.0,2.60e+06,88.94,10.64,0.41,51268,47586,1.08,1.6,4802,274182,1.75,2.2,5.35e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
2,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,vandalism,NaN,1,NaN,NaN,12.12,9.71,6.49,9.28,1586986,1807756,1887927,5284231,30.03,34.21,35.73,2.35e+06,284978.0,9898.0,2.64e+06,88.83,10.79,0.37,53499,49091,1.09,1.9,5226,291955,1.79,2.2,5.46e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
3,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,heavy wind,NaN,3000,NaN,70000.0,10.87,8.19,6.07,8.15,1467293,1801683,1951295,5222116,28.1,34.5,37.37,2.30e+06,276463.0,10150.0,2.59e+06,88.92,10.69,0.39,50447,47287,1.07,2.7,4571,267895,1.71,2.1,5.31e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
4,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,thunderstorm,NaN,2550,NaN,68200.0,11.79,9.25,6.71,9.19,1851519,1941174,1993026,5787064,31.99,33.54,34.44,2.32e+06,278466.0,11010.0,2.61e+06,88.9,10.68,0.42,51598,48156,1.07,0.6,5364,277627,1.93,2.2,5.38e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
5,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 00:00:00,02:00:00,2015-07-19 00:00:00,07:00:00,severe weather,NaN,NaN,1740,250,250000.0,13.07,10.16,7.74,10.43,2028875,2161612,1777937,5970339,33.98,36.21,29.78,2.37e+06,289044.0,9812.0,2.67e+06,88.82,10.81,0.37,54431,49844,1.09,1.7,4873,292023,1.67,2.2,5.49e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48


In [5]:
# DataFrame info
print(df.info())
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nShape: {df.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 1 to 1534
Data columns (total 55 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   YEAR                     1534 non-null   float64
 1   MONTH                    1525 non-null   float64
 2   U.S._STATE               1534 non-null   object 
 3   POSTAL.CODE              1534 non-null   object 
 4   NERC.REGION              1534 non-null   object 
 5   CLIMATE.REGION           1528 non-null   object 
 6   ANOMALY.LEVEL            1525 non-null   object 
 7   CLIMATE.CATEGORY         1525 non-null   object 
 8   OUTAGE.START.DATE        1525 non-null   object 
 9   OUTAGE.START.TIME        1525 non-null   object 
 10  OUTAGE.RESTORATION.DATE  1476 non-null   object 
 11  OUTAGE.RESTORATION.TIME  1476 non-null   object 
 12  CAUSE.CATEGORY           1534 non-null   object 
 13  CAUSE.CATEGORY.DETAIL    1063 non-null   object 
 14  HURRICANE.NAMES         

## Step 2: Data Cleaning and Exploratory Data Analysis
* Clean the data appropriately. For instance, you may need to replace data that should be missing with NaN or create new columns out of given ones (e.g. compute distances, scale data, or get time information from time stamps).
* Look at the distributions of relevant columns separately by using DataFrame operations and drawing at least two relevant plots.
* Look at the statistics of pairs of columns to identify possible associations. For instance, you may create scatter plots and plot conditional distributions, or box-plots. You must plot at least two such plots in your notebook. The results of your bivariate analyses will be helpful in identifying interesting hypothesis tests!
* Choose columns to group and pivot by and examine aggregate statistics.

##### Specific to Dataset
* **Handling the Excel Data:** The data is given as an Excel file rather than a standard CSV. Open the data in Google Sheets or another spreadsheet application first to determine which rows and columns of the sheet should be ignored when loading the data. 
  > **Note:** `pandas` can load multiple filetypes (e.g., `pd.read_csv`, `pd.read_excel`, `pd.read_html`, `pd.read_json`).

* **Parsing Dates and Times:** The power outage start date and time are given by `OUTAGE.START.DATE` and `OUTAGE.START.TIME`. It would be preferable if these two columns were combined into one single `pd.Timestamp` column. 
  * Combine `OUTAGE.START.DATE` and `OUTAGE.START.TIME` into a new column called `OUTAGE.START`. 
  * Similarly, combine `OUTAGE.RESTORATION.DATE` and `OUTAGE.RESTORATION.TIME` into a new column called `OUTAGE.RESTORATION`. 
  > **Tip:** The `pd.to_datetime` and `pd.to_timedelta` functions will be especially useful here.

* **Geospatial Visualization:** To visualize geospatial data, consider `folium` or another geospatial plotting library. You can even embed Folium maps directly into a website. 
  * If `fig` is a `folium.folium.Map` object, calling `fig._repr_html_()` evaluates to a string containing your plot as HTML. 
  * Use Python's built-in `open` and `write` functions to save this string to an `.html` file.

# CHECKPOINT 1:

(6 points) Upload a screenshot of a plotly visualization you’ve created while completing Part 1, Step 2: Data Cleaning and Exploratory Data Analysis.

(6 points) What is the pair of hypotheses you plan on testing in Part 1, Step 4? What is the test statistic you plan on using?

(6 points) What is the column you plan on trying to predict in Part 1, Steps 5-8? Is it a classification or regression problem?


In [6]:
def fix_data_types(df):
    '''
    Fixes data types of columns in the DataFrame based on expected types.
    '''
    datetime_cols = [
        'OUTAGE.START.DATE', 
        #'OUTAGE.START.TIME',       # datetime.time object
        'OUTAGE.RESTORATION.DATE', 
        #'OUTAGE.RESTORATION.TIME'  # datetime.time object
        ]
    int_cols = [
        'YEAR', 'MONTH', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'POPULATION'
        ]
    float_cols = [
        'ANOMALY.LEVEL', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 
        'TOTAL.SALES', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS', 'COM.CUSTOMERS', 'IND.CUSTOMERS', 
        'TOTAL.CUSTOMERS', 'RES.CUST.PCT', 'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA', 
        'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP', 'UTIL.CONTRI', 'PI.UTIL.OFUSA',
        'POPPCT_URBAN', 'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL', 'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 
        'PCT_WATER_TOT', 'PCT_WATER_INLAND'
        ]
    cat_cols = [
        'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION', 'CLIMATE.REGION', 'CLIMATE.CATEGORY', 'CAUSE.CATEGORY', 
        'CAUSE.CATEGORY.DETAIL'
    ]
    # HURRICANE.NAMES will be treated as object automatically 

    for col in datetime_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    for col in int_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
    for col in float_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('float64')
    for col in cat_cols:
        df[col] = df[col].astype('category')
    return df

In [7]:
# Columns with Missing Values
missing_values = df.isnull().sum()
print("\nMissing Values per Column:")
print(missing_values[missing_values > 0].sort_values(ascending=False))


Missing Values per Column:
HURRICANE.NAMES            1462
DEMAND.LOSS.MW              705
CAUSE.CATEGORY.DETAIL       471
CUSTOMERS.AFFECTED          443
OUTAGE.RESTORATION.DATE      58
OUTAGE.RESTORATION.TIME      58
OUTAGE.DURATION              58
RES.SALES                    22
TOTAL.PRICE                  22
IND.PRICE                    22
COM.PRICE                    22
TOTAL.SALES                  22
RES.PRICE                    22
IND.SALES                    22
RES.PERCEN                   22
COM.PERCEN                   22
IND.PERCEN                   22
COM.SALES                    22
POPDEN_UC                    10
POPDEN_RURAL                 10
OUTAGE.START.TIME             9
OUTAGE.START.DATE             9
CLIMATE.CATEGORY              9
ANOMALY.LEVEL                 9
MONTH                         9
CLIMATE.REGION                6
dtype: int64


In [ ]:
# Missing CLIMATE.REGION values --> Hawaii and Alaska, non-continental states
missing_climate_region = df[df['CLIMATE.REGION'].isnull()]
missing_climate_region

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION,MONTH.NAME
1516,2008,12,Hawaii,HI,HI,NaN,-0.7,cold,2008-12-26,18:13:00,2008-12-27,17:00:00,severe weather,thunderstorm,NaN,1367,1060,294000,29.28,26.28,22.43,25.78,253625.0,275383.0,306397.0,835405.0,30.36,32.96,36.68,409668.0,61684.0,673.0,472025.0,86.79,13.07,0.14,50565.0,48401.0,1.04,-0.7,1646.0,67364.0,2.44,0.5,1332213,91.93,20.47,3180.8,1664.7,18.2,6.12,2.60,58.75,41.25,0.38,2008-12-26 18:13:00,2008-12-27 17:00:00,Dec
1517,2011,5,Hawaii,HI,PR,NaN,-0.4,normal,2011-05-02,17:06:00,2011-05-02,20:00:00,severe weather,NaN,NaN,174,220,62000,34.58,32.14,27.85,31.29,249369.0,292304.0,310682.0,852355.0,29.26,34.29,36.45,417531.0,60043.0,698.0,478272.0,87.30,12.55,0.15,49110.0,47586.0,1.03,0.4,1935.0,67686.0,2.86,0.6,1378227,91.93,20.47,3180.8,1664.7,18.2,6.12,2.60,58.75,41.25,0.38,2011-05-02 17:06:00,2011-05-02 20:00:00,May
1518,2006,10,Hawaii,HI,HECO,NaN,0.7,warm,2006-10-15,07:09:00,2006-10-15,16:12:00,severe weather,earthquake,NaN,543,110,59886,23.25,21.26,17.71,20.54,274609.0,307570.0,340735.0,922915.0,29.75,33.33,36.92,401592.0,61334.0,689.0,463615.0,86.62,13.23,0.15,50358.0,48909.0,1.03,1.1,1436.0,65956.0,2.18,0.5,1309731,91.93,20.47,3180.8,1664.7,18.2,6.12,2.60,58.75,41.25,0.38,2006-10-15 07:09:00,2006-10-15 16:12:00,Oct
1519,2006,6,Hawaii,HI,HECO,NaN,0.0,normal,2006-06-01,14:12:00,2006-06-01,18:09:00,system operability disruption,NaN,NaN,237,120,29300,24.09,22.06,18.74,21.45,265474.0,298487.0,327618.0,891580.0,29.78,33.48,36.75,401592.0,61334.0,689.0,463615.0,86.62,13.23,0.15,50358.0,48909.0,1.03,1.1,1436.0,65956.0,2.18,0.5,1309731,91.93,20.47,3180.8,1664.7,18.2,6.12,2.60,58.75,41.25,0.38,2006-06-01 14:12:00,2006-06-01 18:09:00,Jun
1520,2006,10,Hawaii,HI,HECO,NaN,0.7,warm,2006-10-15,07:09:00,2006-10-16,14:55:00,severe weather,earthquake,NaN,1906,1170,291000,23.25,21.26,17.71,20.54,274609.0,307570.0,340735.0,922915.0,29.75,33.33,36.92,401592.0,61334.0,689.0,463615.0,86.62,13.23,0.15,50358.0,48909.0,1.03,1.1,1436.0,65956.0,2.18,0.5,1309731,91.93,20.47,3180.8,1664.7,18.2,6.12,2.60,58.75,41.25,0.38,2006-10-15 07:09:00,2006-10-16 14:55:00,Oct
1534,2000,<NA>,Alaska,AK,ASCC,NaN,NaN,NaN,NaT,NaN,NaT,NaN,equipment failure,failure,NaN,<NA>,35,14273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230534.0,38074.0,854.0,273530.0,84.28,13.92,0.31,57401.0,44745.0,1.28,-2.2,724.0,36046.0,2.01,0.2,627963,66.02,21.56,1802.6,1276.0,0.4,0.05,0.02,85.76,14.24,2.90,NaT,NaT,NaN


In [ ]:
# Missing POPDEN_UC and POPDEN_RURAL values --> District of Columbia
missing_popden_uc_rural = df[df['POPDEN_UC'].isnull() | df['POPDEN_RURAL'].isnull()]
missing_popden_uc_rural

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION,MONTH.NAME
841,2010,2,District of Columbia,DC,RFC,Northeast,1.2,warm,2010-02-05,19:00:00,2010-02-12,15:46:00,severe weather,winter storm,NaN,9886,<NA>,97651,13.41,13.80,7.11,13.55,181006.0,682695.0,18213.0,9.02e+05,20.06,75.65,2.02,227550.0,26449.0,1.0,254001.0,89.59,10.41,3.94e-04,168377.0,47287.0,3.56,0.8,1416.0,101904.0,1.39,0.3,605126,100.0,0.0,9856.5,NaN,NaN,100.0,0.0,89.71,10.29,10.29,2010-02-05 19:00:00,2010-02-12 15:46:00,Feb
842,2011,8,District of Columbia,DC,RFC,Northeast,-0.6,cold,2011-08-27,23:05:00,2011-08-29,15:30:00,severe weather,NaN,NaN,2425,<NA>,220000,13.06,12.77,8.29,12.69,239882.0,801888.0,16927.0,1.08e+06,22.11,73.91,1.56,229450.0,26496.0,1.0,255948.0,89.65,10.35,3.91e-04,167337.0,47586.0,3.52,-0.6,1300.0,103820.0,1.25,0.3,620472,100.0,0.0,9856.5,NaN,NaN,100.0,0.0,89.71,10.29,10.29,2011-08-27 23:05:00,2011-08-29 15:30:00,Aug
843,2010,8,District of Columbia,DC,RFC,Northeast,-1.2,cold,2010-08-05,15:30:00,2010-08-05,22:00:00,severe weather,thunderstorm,NaN,390,<NA>,76729,14.76,13.31,8.13,13.47,236540.0,883096.0,18171.0,1.17e+06,20.27,75.66,1.56,227550.0,26449.0,1.0,254001.0,89.59,10.41,3.94e-04,168377.0,47287.0,3.56,0.8,1416.0,101904.0,1.39,0.3,605126,100.0,0.0,9856.5,NaN,NaN,100.0,0.0,89.71,10.29,10.29,2010-08-05 15:30:00,2010-08-05 22:00:00,Aug
844,2003,9,District of Columbia,DC,RFC,Northeast,0.2,normal,2003-09-18,16:20:00,2003-09-28,18:00:00,severe weather,hurricanes,Isabel,14500,<NA>,530000,8.36,8.61,5.98,8.46,152325.0,721287.0,21491.0,9.21e+05,16.54,78.32,2.33,199215.0,26283.0,1.0,225500.0,88.34,11.66,4.43e-04,155044.0,45858.0,3.38,3.3,1115.0,88143.0,1.26,0.4,568502,100.0,0.0,9856.5,NaN,NaN,100.0,0.0,89.71,10.29,10.29,2003-09-18 16:20:00,2003-09-28 18:00:00,Sep
845,2011,1,District of Columbia,DC,RFC,Northeast,-1.3,cold,2011-01-26,17:00:00,2011-01-31,08:00:00,severe weather,winter storm,NaN,6660,<NA>,210000,13.62,13.17,7.48,13.13,229985.0,744163.0,19395.0,1.02e+06,22.58,73.06,1.90,229450.0,26496.0,1.0,255948.0,89.65,10.35,3.91e-04,167337.0,47586.0,3.52,-0.6,1300.0,103820.0,1.25,0.3,620472,100.0,0.0,9856.5,NaN,NaN,100.0,0.0,89.71,10.29,10.29,2011-01-26 17:00:00,2011-01-31 08:00:00,Jan
846,2013,2,District of Columbia,DC,RFC,Northeast,-0.4,normal,2013-02-08,11:38:00,2013-02-08,14:17:00,equipment failure,generator trip,NaN,159,140,52000,11.97,11.88,5.40,11.73,180750.0,627931.0,15521.0,8.47e+05,21.35,74.16,1.83,235322.0,26530.0,1.0,261854.0,89.87,10.13,3.82e-04,159340.0,48396.0,3.29,-2.5,1165.0,103430.0,1.13,0.3,649540,100.0,0.0,9856.5,NaN,NaN,100.0,0.0,89.71,10.29,10.29,2013-02-08 11:38:00,2013-02-08 14:17:00,Feb
847,2014,1,District of Columbia,DC,RFC,Northeast,-0.5,cold,2014-01-06,19:50:00,2014-01-06,20:44:00,severe weather,winter,NaN,54,<NA>,<NA>,12.59,12.67,8.66,12.49,218682.0,736898.0,20478.0,1.00e+06,21.76,73.33,2.04,239355.0,26463.0,1.0,265820.0,90.04,9.96,3.76e-04,159831.0,49091.0,3.26,0.3,1107.0,105312.0,1.05,0.3,659836,100.0,0.0,9856.5,NaN,NaN,100.0,0.0,89.71,10.29,10.29,2014-01-06 19:50:00,2014-01-06 20:44:00,Jan
848,2013,6,District of Columbia,DC,RFC,Northeast,-0.2,normal,2013-06-13,15:30:00,2013-06-13,16:00:00,severe weather,uncontrolled loss,NaN,30,700,40000,12.65,11.77,5.33,11.72,175324.0,789494.0,20375.0,1.02e+06,17.26,77.72,2.01

In [18]:
# Show duplicates
df_duplicates = df[df.duplicated(keep=False)]
df_duplicates

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION,MONTH.NAME
25,2014,1,Tennessee,TN,SERC,Central,-0.5,cold,2014-01-07,06:00:00,2014-01-07,08:30:00,severe weather,winter,NaN,150,<NA>,<NA>,9.73,10.02,6.27,9.13,4.80e+06,2.84e+06,1.91e+06,9.55e+06,50.21,29.76,20.03,2.76e+06,4.72e+05,1271.0,3.23e+06,85.35,14.61,0.04,41955.0,49091.0,0.85,0.8,1739.0,2.75e+05,0.63,0.4,6547779,66.39,12.02,1450.3,1076.2,55.6,7.05,1.72,97.84,2.16,2.16,2014-01-07 06:00:00,2014-01-07 08:30:00,Jan
44,2014,1,Tennessee,TN,SERC,Central,-0.5,cold,2014-01-07,06:00:00,2014-01-07,08:30:00,severe weather,winter,NaN,150,<NA>,<NA>,9.73,10.02,6.27,9.13,4.80e+06,2.84e+06,1.91e+06,9.55e+06,50.21,29.76,20.03,2.76e+06,4.72e+05,1271.0,3.23e+06,85.35,14.61,0.04,41955.0,49091.0,0.85,0.8,1739.0,2.75e+05,0.63,0.4,6547779,66.39,12.02,1450.3,1076.2,55.6,7.05,1.72,97.84,2.16,2.16,2014-01-07 06:00:00,2014-01-07 08:30:00,Jan
220,2013,12,Texas,TX,TRE,South,-0.3,normal,2013-12-13,11:00:00,2013-12-27,11:00:00,fuel supply emergency,Coal,NaN,20160,<NA>,<NA>,11.27,8.00,5.79,8.67,1.19e+07,1.07e+07,8.27e+06,3.09e+07,38.50,34.74,26.76,9.95e+06,1.37e+06,95881.0,1.14e+07,87.19,11.97,0.84,51695.0,48396.0,1.07,2.7,28880.0,1.37e+06,2.11,10.5,26500674,84.70,9.35,2435.3,1539.9,15.2,3.35,0.58,97.26,2.74,2.09,2013-12-13 11:00:00,2013-12-27 11:00:00,Dec
274,2013,12,Texas,TX,TRE,South,-0.3,normal,2013-12-13,11:00:00,2013-12-27,11:00:00,fuel supply emergency,Coal,NaN,20160,<NA>,<NA>,11.27,8.00,5.79,8.67,1.19e+07,1.07e+07,8.27e+06,3.09e+07,38.50,34.74,26.76,9.95e+06,1.37e+06,95881.0,1.14e+07,87.19,11.97,0.84,51695.0,48396.0,1.07,2.7,28880.0,1.37e+06,2.11,10.5,26500674,84.70,9.35,2435.3,1539.9,15.2,3.35,0.58,97.26,2.74,2.09,2013-12-13 11:00:00,2013-12-27 11:00:00,Dec
500,2004,7,Arizona,AZ,WECC,Southwest,0.5,warm,2004-07-06,06:00:00,2004-08-09,12:00:00,severe weather,wildfire,NaN,49320,<NA>,<NA>,8.88,7.57,5.55,7.91,3.47e+06,2.56e+06,1.05e+06,7.08e+06,48.97,36.22,14.81,2.26e+06,2.59e+05,7419.0,2.52e+06,89.44,10.26,0.29,41169.0,47037.0,0.88,1.0,5113.0,2.33e+05,2.20,1.7,5652404,89.81,9.74,2625.4,1669.0,5.8,1.92,0.33,99.65,0.35,0.35,2004-07-06 06:00:00,2004-08-09 12:00:00,Jul
514,2004,7,Arizona,AZ,WECC,Southwest,0.5,warm,2004-07-06,06:00:00,2004-08-09,12:00:00,severe weather,wildfire,NaN,49320,<NA>,<NA>,8.88,7.57,5.55,7.91,3.47e+06,2.56e+06,1.05e+06,7.08e+06,48.97,36.22,14.81,2.26e+06,2.59e+05,7419.0,2.52e+06,89.44,10.26,0.29,41169.0,47037.0,0.88,1.0,5113.0,2.33e+05,2.20,1.7,5652404,89.81,9.74,2625.4,1669.0,5.8,1.92,0.33,99.65,0.35,0.35,2004-07-06 06:00:00,2004-08-09 12:00:00,Jul
994,2010,8,Louisiana,LA,SERC,South,-1.2,cold,2010-08-02,12:45:00,2010-08-04,11:00:00,public appeal,NaN,NaN,2775,<NA>,<NA>,9.29,8.50,5.99,8.13,3.70e+06,2.41e+06,2.40e+06,8.51e+06,43.45,28.31,28.23,1.97e+06,2.74e+05,18128.0,2.27e+06,87.10,12.10,0.80,48305.0,47287.0,1.02,3.6,4509.0,2.20e+05,2.05,1.4,4544951,73.19,11.85,1685.8,1280.1,29.5,4.56,0.97,82.49,17.51,8.71,2010-08-02 12:45:00,2010-08-04 11:00:00,Aug
998,2010,8,Louisiana,LA,SERC,South,-1.2,cold,2010-08-02,12:45:00,2010-08-04,11:00:00,public appeal,NaN,NaN,2775,<NA>,<NA>,9.29,8.50,5.99,8.13,3.70e+06,2.41e+06,2.40e+06,8.51e+06,43.45,28.31,28.23,1.97e+06,2.74e+05,18128.0,2.27e+06,87.10,12.10,0.80,48305.0,47287.0,1.02,3.6,4509.0,2.20e+05,2.05,1.4,4544951,73.19,11.85,16

In [19]:
# Data Cleaning Functions
def remove_duplicate_rows(df):
    """
    Remove duplicate rows 
    """
    df = df.drop_duplicates()
    return df

def combine_outage_start(df): 
    """
    Combine OUTAGE.START.DATE and OUTAGE.START.TIME into a single column OUTAGE.START.
    """
    time_as_td = pd.to_timedelta(df['OUTAGE.START.TIME'].astype(str), errors='coerce')
    df['OUTAGE.START'] = df['OUTAGE.START.DATE'] + time_as_td
    return df

def combine_outage_restoration(df): 
    """
    Combine OUTAGE.RESTORATION.DATE and OUTAGE.RESTORATION.TIME into a single column OUTAGE.RESTORATION.
    """
    time_as_td = pd.to_timedelta(df['OUTAGE.RESTORATION.TIME'].astype(str), errors='coerce')
    df['OUTAGE.RESTORATION'] = df['OUTAGE.RESTORATION.DATE'] + time_as_td
    return df

def add_month_names(df):
    """
    Map numeric MONTH to MONTH.NAME.
    """
    MONTH_NAMES = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',
                   7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
    df['MONTH.NAME'] = df['MONTH'].map(MONTH_NAMES)
    return df


In [21]:
# Apply all cleaning functions in a pipeline

df_cleaned = (
    df.pipe(remove_duplicate_rows)
      .pipe(fix_data_types)
      .pipe(combine_outage_start)
      .pipe(combine_outage_restoration)
      .pipe(add_month_names)
)

In [22]:
df_cleaned.head()

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION,MONTH.NAME
1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,2011-07-03,20:00:00,severe weather,NaN,NaN,3060,<NA>,70000,11.60,9.18,6.81,9.28,2.33e+06,2.11e+06,2.11e+06,6.56e+06,35.55,32.23,32.20,2.31e+06,276286.0,10673.0,2.60e+06,88.94,10.64,0.41,51268.0,47586.0,1.08,1.6,4802.0,274182.0,1.75,2.2,5348119,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.59,8.41,5.48,2011-07-01 17:00:00,2011-07-03 20:00:00,Jul
2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,2014-05-11,18:39:00,intentional attack,vandalism,NaN,1,<NA>,<NA>,12.12,9.71,6.49,9.28,1.59e+06,1.81e+06,1.89e+06,5.28e+06,30.03,34.21,35.73,2.35e+06,284978.0,9898.0,2.64e+06,88.83,10.79,0.37,53499.0,49091.0,1.09,1.9,5226.0,291955.0,1.79,2.2,5457125,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.59,8.41,5.48,2014-05-11 18:38:00,2014-05-11 18:39:00,May
3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,2010-10-28,22:00:00,severe weather,heavy wind,NaN,3000,<NA>,70000,10.87,8.19,6.07,8.15,1.47e+06,1.80e+06,1.95e+06,5.22e+06,28.10,34.50,37.37,2.30e+06,276463.0,10150.0,2.59e+06,88.92,10.69,0.39,50447.0,47287.0,1.07,2.7,4571.0,267895.0,1.71,2.1,5310903,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.59,8.41,5.48,2010-10-26 20:00:00,2010-10-28 22:00:00,Oct
4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,04:30:00,2012-06-20,23:00:00,severe weather,thunderstorm,NaN,2550,<NA>,68200,11.79,9.25,6.71,9.19,1.85e+06,1.94e+06,1.99e+06,5.79e+06,31.99,33.54,34.44,2.32e+06,278466.0,11010.0,2.61e+06,88.90,10.68,0.42,51598.0,48156.0,1.07,0.6,5364.0,277627.0,1.93,2.2,5380443,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.59,8.41,5.48,2012-06-19 04:30:00,2012-06-20 23:00:00,Jun
5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,02:00:00,2015-07-19,07:00:00,severe weather,NaN,NaN,1740,250,250000,13.07,10.16,7.74,10.43,2.03e+06,2.16e+06,1.78e+06,5.97e+06,33.98,36.21,29.78,2.37e+06,289044.0,9812.0,2.67e+06,88.82,10.81,0.37,54431.0,49844.0,1.09,1.7,4873.0,292023.0,1.67,2.2,5489594,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.59,8.41,5.48,2015-07-18 02:00:00,2015-07-19 07:00:00,Jul


In [ ]:
# Export cleaned dataset to CSV
output_path = 'data/cleaned_outage_dataset.csv'
df_cleaned.to_csv(output_path, index=False)
print(f"Cleaned dataset exported to {output_path}")

# Exploratory Data Analysis

In [81]:
pio.renderers.default = "vscode" 

In [82]:
# Outage Frequency over the Years, also Outage Frequency by Month
yearly = df_cleaned.groupby('YEAR').size()
yearly.plot(kind='line', title='Number of Outages per Year').show()

month_order = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
by_month = df_cleaned.groupby('MONTH.NAME').size().reindex(month_order)
by_month.plot(kind='bar', title='Number of Outages by Month (all years combined)').show()


In [83]:
# Regional Analysis
by_region = df_cleaned.groupby('CLIMATE.REGION', observed=True).size().sort_values(ascending=False)
by_region.plot(kind='bar', title='Number of Outages by Climate Region').show()

top_15_states = df_cleaned['U.S._STATE'].value_counts().nlargest(15).sort_values(ascending=True)
top_15_states.plot(kind='barh', title='Top 15 States by Number of Outages').show()

In [84]:
# Cause Analysis
by_cause = df_cleaned.groupby('CAUSE.CATEGORY', observed=True).size().sort_values(ascending=True)
by_cause.plot(kind='barh', title='Number of Outages by Cause Category').show()

# Average Outage Duration by Cause Category
avg_duration_by_cause = df_cleaned.groupby('CAUSE.CATEGORY', observed=True)['OUTAGE.DURATION'].mean().sort_values(ascending=False)
avg_duration_by_cause.plot(kind='bar', title='Average Outage Duration by Cause Category').show()

## Step 3: Assessment of Missingness

In [68]:
# TODO

## Step 4: Hypothesis Testing

# Three options for hypothesis testing:

## Comparing Outage Durations by Cause (Permutation Test)

Our EDA shows that different causes lead to different outage times. We can test if this difference is statistically significant.

Null Hypothesis (H0): The distribution of outage durations is the same for outages caused by "severe weather" and "equipment failure." Any observed difference in sample means is due to random chance.

Alternative Hypothesis (H1): Outages caused by "severe weather" have a longer average duration than outages caused by "equipment failure."

Test Statistic: Difference in group means (Mean Duration of Severe Weather - Mean Duration of Equipment Failure).

## Climate Categories and Outage Causes (Chi-Square or TVD)

You have a CLIMATE.CATEGORY column (warm, cold, normal) and a CAUSE.CATEGORY column. You could see if the types of outages that occur depend on the climate conditions.

Null Hypothesis (H0): The distribution of outage causes is independent of the climate category (warm vs. cold).

Alternative Hypothesis (H1): The distribution of outage causes depends on the climate category (e.g., certain causes are more likely in cold climates vs. warm climates).

Test Statistic: Total Variation Distance (TVD) if you are comparing exactly two distributions, or the Chi-Square statistic if comparing across multiple categories.

## Customers Affected by Region (Permutation Test)

You can compare the impact of outages across different NERC or Climate regions.

Null Hypothesis (H0): Outages in the "Northeast" climate region affect the same number of customers, on average, as outages in the "South" climate region.

Alternative Hypothesis (H1): Outages in the "Northeast" climate region affect a different number of customers, on average, compared to the "South" climate region.

Test Statistic: Absolute difference in sample means ∣Mean Customers Northeast −Mean Customers South ∣. (Using the absolute difference makes this a two-sided test).

Test Statistic: Absolute difference in sample means ∣Mean Customers Northeast −Mean Customers South ∣. (Using the absolute difference makes this a two-sided test).

In [1]:
# TODO

## Step 5: Framing a Prediction Problem

In [70]:
# TODO

## Step 6: Baseline Model

In [71]:
# TODO

## Step 7: Final Model

In [72]:
# TODO

## Step 8: Fairness Analysis

In [73]:
# TODO